In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
import pandas as pd
from langchain_classic.chains import RetrievalQA
from langchain_classic.agents import initialize_agent, AgentType, Tool
from langchain_core.documents import Document
import re
import numpy as np
from datetime import datetime
from datetime import time as time_class


# Problem Classification

In [ ]:
def build_llm(model: str = "gpt-oss:20b", temperature: float = 0.0) -> ChatOllama:
    return ChatOllama(
        model=model,
        temperature=temperature,
        base_url="http://localhost:11434",   
        timeout=120                          
    )

llm1 = build_llm()         
llm2 = build_llm()       
llm3 = build_llm()     
llm_code = build_llm()    
embeddings = OllamaEmbeddings(model="nomic-embed-text")

def Classification_Agent(file_path="Large_Scale_Or_Files/RefData.csv"):
    # Load and process the data
    loader1 = CSVLoader(file_path="Large_Scale_Or_Files/RefData.csv", encoding="utf-8")
    refdata = loader1.load()
    refdocuments = refdata

    vectors1 = FAISS.from_documents(refdocuments, embeddings)
    retriever1 = vectors1.as_retriever(search_kwargs={'k': 5})
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm1,
        chain_type="stuff",
        retriever=retriever1,
        return_source_documents=True,
    )

    qa_tool1 = Tool(
        name="FileQA",
        func=qa_chain.invoke,
        description=(
            "Use this tool to answer questions about the problem type of the text. "
            "Provide the question as input, and the tool will retrieve the relevant information from the file and use it to answer the question."
        ),
    )

    few_shot_examples = """

    Question: What is the problem type in operation of the text? Please give the answer directly. Text:There are three best-selling items (P1, P2, P3) on Amazon with the profit w_1,w_2,w_3.There is an independent demand stream for each of the products. The objective of the company is to decide which demands to be fufilled over a ﬁnite sales horizon [0,10] to maximize the total expected revenue from ﬁxed initial inventories. The on-hand inventories for the three items are c_1,c_2,c_3 respectively. During the sales horizon, replenishment is not allowed and there is no any in-transit inventories. Customers who want to purchase P1,P2,P3 arrive at each period accoring to a Poisson process with a_1,a_2,a_3 the arrival rates respectively. Decision variables y_1,y_2,y_3 correspond to the number of requests that the firm plans to fulfill for product 1,2,3. These variables are all positive integers.

    Thought: I need to determine the problem type of the text. I'll use the FileQA tool to retrieve the relevant information.

    Action: FileQA

    Action Input: "What is the problem type in operation of the text? text:There are three best-selling items (P1, P2, P3) on Amazon with the profit w_1, w_2, w_3. ..."

    Observation: The problem type of the text is Network Revenue Management.

    Final Answer: Network Revenue Management.

    """

    prefix = f"""You are a helpful assistant that can answer questions about operation problems.

    Use the following examples as a guide. Always use the FileQA tool when you need to retrieve information from the file:


    {few_shot_examples}

    When you need to find information from the file, use the provided tools. And answer the question by given the answer directly. For example,

    """

    suffix = """

    Begin!

    Question: {input}
    {agent_scratchpad}"""

    agent_pc = initialize_agent(
        tools=[qa_tool1],
        llm=llm1,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        agent_kwargs={
            "prefix": prefix.format(few_shot_examples=few_shot_examples),
            "suffix": suffix,
        },
        verbose=True,
        handle_parsing_errors=True,
    )
    return agent_pc

def Problemtype(query):
    agent_pc = Classification_Agent(file_path="Large_Scale_Or_Files/RefData.csv")
    category_original=agent_pc.invoke(f"What is the problem type in operation of the text? text:{query}")
    type_output = category_original['output']
    return type_output

# ProblemSolvingModel

## PreProcess: Sharing Functions

In [ ]:
def LoadFiles():
  v1 = pd.read_csv('Test_Dataset/Air_NRM/v1.csv')
  v2 = pd.read_csv('Test_Dataset/Air_NRM/v2.csv')
  demand = pd.read_csv('Test_Dataset/Air_NRM/od_demand.csv')
  flight = pd.read_csv('Test_Dataset/Air_NRM/flight.csv')
  return v1,v2,demand,flight

In [ ]:
def New_Vectors_Flight(query):
  v1,v2,demand,df = LoadFiles()
  new_docs = []
  for _, row in df.iterrows():
      new_docs.append(Document(
          page_content=f"OD={row['Oneway_OD']},Departure_Time_Flight1={row['Departure Time']},Oneway_Product={row['Oneway_Product']},avg_price={row['Avg Price']}",
          metadata={
              'OD': row['Oneway_OD'],
              'time': row['Departure Time'],
              'product': row['Oneway_Product'],
              'avg_price': row['Avg Price']
          }
      ))

  new_vectors = FAISS.from_documents(new_docs, embeddings)
  return new_vectors

def New_Vectors_Demand(query):
  v1,v2,demand,df = LoadFiles()
  new_docs = []
  for _, row in demand.iterrows():
      new_docs.append(Document(
          page_content=f"OD={row['Oneway_OD']}, avg_pax={row['Avg Pax']}",
          metadata={
              'OD': row['Oneway_OD'],
              'avg_pax': row['Avg Pax']
          }
      ))

  new_vectors = FAISS.from_documents(new_docs, embeddings)
  return new_vectors


def retrieve_key_information(query):
    flight_pattern = r"\(OD\s*=\s*\(\s*'(\w+)'\s*,\s*'(\w+)'\s*\)\s+AND\s+Departure\s+Time='(\d{1,2}:\d{2})'\)"
    matches = re.findall(flight_pattern, query)

    flight_strings = [f"(OD = ('{origin}', '{destination}') AND Departure Time='{departure_time}')" for origin, destination, departure_time in matches]

    new_query = "Retrieve avg_price, avg_pax, value_list, ratio_list, value_0_list, ratio_0_list, and flight_capacity for the following flights:\n" + ", ".join(flight_strings) + "."

    return new_query
def retrieve_time_period(departure_time):
    intervals = {
        '12pm~6pm': (time_class(12, 0), time_class(18, 0)),
        '6pm~10pm': (time_class(18, 0), time_class(22, 0)),
        '10pm~8am': (time_class(22, 0), time_class(8, 0)),
        '8am~12pm': (time_class(8, 0), time_class(12, 0))
    }

    if isinstance(departure_time, str):
        try:
            hours, minutes = map(int, departure_time.split(':'))
            departure_time = time_class(hours, minutes)
        except ValueError:
            raise ValueError("Time format should be: 'HH:MM'")

    for interval_name, (start, end) in intervals.items():
        if start < end:
            if start <= departure_time < end:
                return interval_name
        else:
            if departure_time >= start or departure_time < end:
                return interval_name

    return "Unknown"
def retrieve_parameter(O,time_interval,product):
    v1,v2,df,demand = LoadFiles()
    time_interval = f'({time_interval})'
    key = product + '*' + time_interval
    _value_ = 0
    _ratio_ = 0
    no_purchase_value = 0
    no_purchase_value_ratio = 0
    subset = v1[v1['OD Pairs'] == O]
    if key in subset.columns and not subset.empty:
        _value_ = subset[key].values[0]

    subset2 = v2[v2['OD Pairs'] == O]
    if key in subset2.columns and not subset2.empty:
        _ratio_ = subset2[key].values[0]

    if 'no_purchase' in subset.columns and not subset.empty:
        no_purchase_value = subset['no_purchase'].values[0]

    if 'no_purchase' in subset2.columns and not subset2.empty:
        no_purchase_value_ratio = subset2['no_purchase'].values[0]
    return _value_,_ratio_,no_purchase_value,no_purchase_value_ratio

def generate_coefficients(OD,time):
    departure_time = datetime.strptime(time, '%H:%M').time()
    time_interval = retrieve_time_period(departure_time)
    value_1,ratio_1,value_0,ratio_0 = retrieve_parameter(OD,time_interval,'Eco_flexi')

    value_2,ratio_2,value_0,ratio_0 = retrieve_parameter(OD,time_interval,'Eco_lite')

    return value_1,ratio_1,value_2,ratio_2,value_0,ratio_0


def clean_text_preserve_newlines(text):
    cleaned = re.sub(r'\x1b\[[0-9;]*[mK]', '', text)
    cleaned = re.sub(r'[^\x20-\x7E\n]', '', cleaned)
    cleaned = re.sub(r'(\n\s+)(\w+\s*=)', r'\n\2', cleaned)
    cleaned = re.sub(r'\[\s+', '[', cleaned)
    cleaned = re.sub(r'\s+\]', ']', cleaned)
    cleaned = re.sub(r',\s+', ', ', cleaned)

    return cleaned


## Flow

In [ ]:
def csv_qa_tool_flow(query: str,raw_query):
    new_vectors = New_Vectors_Flight(query)
    matches = re.findall(r"\(OD\s*=\s*(\(\s*'[^']+'\s*,\s*'[^']+'\s*\))\s+AND\s+Departure\s*Time\s*=\s*'(\d{1,2}:\d{2})'\)", query)
    num_match = re.search(r"optimal\s+(\d+)\s+flights", raw_query, re.IGNORECASE)
    num_flights = int(num_match.group(1)) if num_match else None
    capacity_match = re.search(r"Eco_flex ticket consumes (\d+\.?\d*)\s*units", raw_query)
    if matches == []:
        pattern = r"\(\('(\w+)','(\w+)'\),\s*'(\d{1,2}:\d{2})'\)"
        matches_2 = re.findall(pattern, query)

    if capacity_match:
        eco_flex_capacity = capacity_match.group(1)
    else:
        eco_flex_capacity = 1.2

    sigma_inflow_A = []
    sigma_outflow_A = []
    sigma_inflow_B = []
    sigma_outflow_B = []
    sigma_inflow_C = []
    sigma_outflow_C = []

    if matches == []:
        matches = matches_2
        for origin,destination,time in matches:
            if origin == 'A':
                flight_name = f"({origin}{destination},{time})"
                sigma_inflow_A.append(flight_name)
            elif origin == 'B':
                flight_name = f"({origin}{destination},{time})"
                sigma_inflow_B.append(flight_name)
            elif origin == 'C':
                flight_name = f"({origin}{destination},{time})"
                sigma_inflow_C.append(flight_name)
            elif destination == 'A':
                flight_name = f"({origin}{destination},{time})"
                sigma_outflow_A.append(flight_name)
            elif destination == 'B':
                flight_name = f"({origin}{destination},{time})"
                sigma_outflow_B.append(flight_name)
            elif destination == 'C':
                flight_name = f"({origin}{destination},{time})"
                sigma_outflow_C.append(flight_name)
    
    else:
        a_origin_flights_A_out = [
            (od, time)
            for (od, time) in matches
            if od[2] == 'A'
        ]

        a_origin_flights_B_out = [
            (od, time)
            for (od, time) in matches
            if od[2] == 'B'
        ]

        a_origin_flights_C_out = [
            (od, time)
            for (od, time) in matches
            if od[2] == 'C'
        ]

        a_origin_flights_A = [
            (od, time)
            for (od, time) in matches
            if od[7] == 'A'
        ]

        a_origin_flights_B = [
            (od, time)
            for (od, time) in matches
            if od[7] == 'B'
        ]

        a_origin_flights_C = [
            (od, time)
            for (od, time) in matches
            if od[7] == 'C'
        ]

        for od, time in a_origin_flights_A:
            origin = od[2]
            destination = od[7]
            flight_name = f"({origin}{destination},{time})"
            sigma_inflow_A.append(flight_name)

        for od, time in a_origin_flights_B:
            origin = od[2]
            destination = od[7]
            flight_name = f"({origin}{destination},{time})"
            sigma_inflow_B.append(flight_name)

        for od, time in a_origin_flights_C:
            origin = od[2]
            destination = od[7]
            flight_name = f"({origin}{destination},{time})"
            sigma_inflow_C.append(flight_name)

        for od, time in a_origin_flights_A_out:
            origin = od[2]
            destination = od[7]
            flight_name = f"({origin}{destination},{time})"
            sigma_outflow_A.append(flight_name)

        for od, time in a_origin_flights_B_out:
            origin = od[2]
            destination = od[7]
            flight_name = f"({origin}{destination},{time})"
            sigma_outflow_B.append(flight_name)

        for od, time in a_origin_flights_C_out:
            origin = od[2]
            destination = od[7]
            flight_name = f"({origin}{destination},{time})"
            sigma_outflow_C.append(flight_name)

    avg_price, x, x_o, ratio_0_list, ratio_list, value_list, value_0_list, avg_pax = {}, {}, {}, {}, {}, {}, {}, {}
    y = {}
    N_l = {"AB":[],"AC":[],"BA":[],"CA":[]}

    if matches == []:
        matches = matches_2
        for origin,destination,time in matches:
            od = str((origin, destination))
            code_f = f"({origin}{destination},{time},f)"
            code_l = f"({origin}{destination},{time},l)"
            code_o = f"{origin}{destination}"
            x[code_f] = f"x_{origin}{destination}_{time}_f"
            x[code_l] = f"x_{origin}{destination}_{time}_l"
            code_y = f"({origin}{destination},{time})"
            y[code_y] = f"y_{origin}{destination}_{time}"
            retriever = new_vectors.as_retriever(search_kwargs={'k': 1,"filter": {"OD": od, "time": time}})

            doc_1 = retriever.invoke(f"OD={od}, Departure Time={time}, Oneway_Product=Eco_flexi, avg_price=")

            for doc in doc_1:
                content = doc.page_content
                pattern = r',\s*(?=\w+=)'
                parts = re.split(pattern, content)

                pairs = [p.strip().replace('"', "'") for p in parts]
                for pair in pairs:
                    key, value = pair.split('=')
                    if key == 'avg_price':
                        avg_price[code_f] = value


            doc_2= retriever.invoke(f"OD={od}, Departure Time={time}, Oneway_Product=Eco_lite, avg_price=")
            for doc in doc_2:
                content = doc.page_content
                pattern = r',\s*(?=\w+=)'
                parts = re.split(pattern, content)

                pairs = [p.strip().replace('"', "'") for p in parts]
                for pair in pairs:
                    key, value = pair.split('=')
                    if key == 'avg_price':
                        avg_price[code_l] = value

            value_1,ratio_1,value_2,ratio_2,value_0,ratio_0 = generate_coefficients(od,time)

            ratio_list[code_f] = ratio_1
            ratio_list[code_l] = ratio_2
            value_list[code_f] = value_1
            value_list[code_l] = value_2
            ratio_0_list[code_o] = ratio_0
            value_0_list[code_o] = value_0
    else:
        for match in matches:
            origin = match[0][2]
            destination = match[0][7]
            time = match[1]
            od = str((origin, destination))
            code_f = f"({origin}{destination},{time},f)"
            code_l = f"({origin}{destination},{time},l)"
            code_o = f"{origin}{destination}"
            x[code_f] = f"x_{origin}{destination}_{time}_f"
            x[code_l] = f"x_{origin}{destination}_{time}_l"
            code_y = f"({origin}{destination},{time})"
            y[code_y] = f"y_{origin}{destination}_{time}"
            retriever = new_vectors.as_retriever(search_kwargs={'k': 1,"filter": {"OD": od, "time": time}})

            doc_1= retriever.invoke(f"OD={od}, Departure Time={time}, Oneway_Product=Eco_flexi, avg_price=")
            for doc in doc_1:
                content = doc.page_content
                pattern = r',\s*(?=\w+=)'
                parts = re.split(pattern, content)

                pairs = [p.strip().replace('"', "'") for p in parts]
                for pair in pairs:
                    key, value = pair.split('=')
                    if key == 'avg_price':
                        avg_price[code_f] = value


            doc_2= retriever.invoke(f"OD={od}, Departure Time={time}, Oneway_Product=Eco_lite, avg_price=")
            for doc in doc_2:
                content = doc.page_content
                pattern = r',\s*(?=\w+=)'
                parts = re.split(pattern, content)

                pairs = [p.strip().replace('"', "'") for p in parts]
                for pair in pairs:
                    key, value = pair.split('=')
                    if key == 'avg_price':
                        avg_price[code_l] = value

            value_1,ratio_1,value_2,ratio_2,value_0,ratio_0 = generate_coefficients(od,time)

            ratio_list[code_f] = ratio_1
            ratio_list[code_l] = ratio_2
            value_list[code_f] = value_1
            value_list[code_l] = value_2
            ratio_0_list[code_o] = ratio_0
            value_0_list[code_o] = value_0


    od_matches = re.findall(
    r"OD\s*=\s*\(\s*'([^']+)'\s*,\s*'([^']+)'\s*\)", 
    query
    )
    if od_matches == []:
        pattern = r"\(\('(\w+)','(\w+)'\)"
        matches = re.findall(pattern, query)
        od_matches = matches
    
    od_matches = list(set(od_matches))

    new_vectors_demand = New_Vectors_Demand(query)
    for origin, dest in od_matches:
        od = str((origin, dest))
        code_o = f"{origin}{dest}"
        x_o[code_o] = f"x_{origin}{dest}_o"
        retriever = new_vectors_demand.as_retriever(search_kwargs={'k': 1})
    
        doc_1= retriever.invoke(f"OD={od}, avg_pax=")
        content = doc_1[0].page_content

        pattern = r',\s*(?=\w+=)'
        parts = re.split(pattern, content)
        pairs = [p.strip().replace('"', "'") for p in parts]

        for pair in pairs:
            key, value = pair.split('=')
            if key == 'avg_pax':
                avg_pax[code_o] = value
            
    doc = f"y = {y}\n"
    doc += f"avg_price={avg_price}\n"
    doc += f"value_list ={value_list}\n"
    doc += f"ratio_list={ratio_list}\n"
    doc += f"value_0_list={value_0_list}\n"
    doc += f"ratio_0_list={ratio_0_list}\n"
    doc += f"avg_pax={avg_pax}\n"
    doc += f"capacity_consum={eco_flex_capacity}\n"
    doc += f"option_num={num_flights}\n"
    doc += f"sigma_inflow_A={sigma_inflow_A}\n"
    doc += f"sigma_outflow_A={sigma_outflow_A}\n"
    doc += f"sigma_inflow_B={sigma_inflow_B}\n"
    doc += f"sigma_outflow_B={sigma_outflow_B}\n"
    doc += f"sigma_inflow_C={sigma_inflow_C}\n"
    doc += f"sigma_outflow_C={sigma_outflow_C}\n"
    doc += f"M = 10000000\n"
    doc += f"flight_capacity=187\n"

    return doc

def retrieve_similar_docs(query,retriever):
    
    similar_docs = retriever.invoke(query)

    results = []
    for doc in similar_docs:
        results.append({
            "content": doc.page_content,
            "metadata": doc.metadata
        })
    return results

def final_tool_flow(problem_description):
    example_matches = retrieve_key_information(problem_description)
    example_data_description = csv_qa_tool_flow(example_matches,problem_description) 
    example_data_description = example_data_description.replace('np.float64', 'float')
    example_data_description = example_data_description.replace('np.int64', 'int') 
    return example_data_description

def FlowAgent(query):
    loader = CSVLoader(file_path="Large_Scale_Or_Files/RAG_Example_SBLP_Flow.csv", encoding="utf-8")
    data = loader.load()
    documents = data
    vectors = FAISS.from_documents(documents, embeddings)
    retriever = vectors.as_retriever(search_kwargs={'k': 1})
    similar_results = retrieve_similar_docs(query,retriever)
    problem_description = similar_results[0]['content'].replace("prompt:", "").strip()  
    example_data_description = final_tool_flow(problem_description)

    fewshot_example = f'''
    Question: {problem_description}

    Thought: I need to retrive the required OD and Departure Time information.

    Action: CSVQA

    Action Input: {problem_description}

    Observation: {example_data_description}

    Thought: I have successfully received the data. I must now return it.
    Final Answer: {example_data_description}
    '''
    tools = [Tool(name="CSVQA", func=final_tool_flow, description="Retrieve flight data.")]
    fewshot_example = fewshot_example.replace('{', '{{')
    fewshot_example = fewshot_example.replace('}', '}}')    
    prefix = (
f"{fewshot_example}\n\n"
"You MUST follow EXACTLY the pattern below for the **FIRST** response.\n"
"Output **ONLY** the following three lines:\n\n"
"Thought: I must first use the CSVQA tool to retrieve necessary data based on the user query.\n"
"Action: CSVQA\n"
"Action Input: {input}\n"
)

    suffix = (
"### FINAL STEP\n"
"Use the latest Observation above.\n"
"Output exactly two lines:\n"
"Thought: Data retrieval complete. I will now return it.\n"
"Final Answer: <copy the Observation content above EXACTLY as is>"
)
    agent2 = initialize_agent(
        tools,
        llm=llm2,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        agent_kwargs={
            "prefix": prefix,
            "suffix": suffix,
        },
        verbose=True,
        handle_parsing_errors=True
    )

    return agent2

def policy_sblp_flow_model_code(query):
    query = query.replace('{', '{{')
    query = query.replace('}', '}}')    
    agent2 = FlowAgent(query)
    result = agent2.invoke({"input": query})
    example_data_description = result['output']
    final_answer = f'''### Objective Function

\[
\max \quad \sum_(l,k,j) avg\_price[(l,k,j)] \cdot x[(l,k,j)]
\]

### Constraints

#### 1. Capacity Constraints

For each flight (OD pair \( l \), departure time \( k \)):

\[
capacity\_consum \cdot x[(l,k,f)] + x[(l,k,l)] \leq flight\_capacity
\]

#### 2. Balance Constraints

For each OD pair \( l \):

\[
ratio\_0\_list[l] \cdot x_o[l] + \sum_k \sum_j ratio\_list[(l,k,j)] \cdot x[(l,k,j)] = avg\_pax[l]
\]

#### 3. Scale Constraints

For each ticket option \((l,k,j)\):

\[
x[(l,k,j)]/value\_list[(l,k,j)] - x_o[l]/value\_0\_list[l] \leq 0
\]

#### 4. Big M Constraints

For each ticket option \((l,k,j)\):

\[
x[(l,k,j)] \leq 10000000 \cdot y[(l,k)]
\]

#### 5. Cardinality Constraint

\[
\sum_(l,k) y[(l,k)] \leq option\_num
\]


#### 6. Flow Conservation Constraints

\[
\sum_(l,k)  \in sigma_inflow_A y[(l,k)] = \sum_(l,k)  \in sigma_outflow_A y[(l,k)]
\]
\[
\sum_(l,k)  \in sigma_inflow_B y[(l,k)] = \sum_(l,k)  \in sigma_outflow_B y[(l,k)]
\]
\[
\sum_(l,k)  \in sigma_inflow_C y[(l,k)] = \sum_(l,k)  \in sigma_outflow_C y[(l,k)]
\]

#### 7. Nonnegativity Constraints

\[
x[(l,k,j)] \geq 0, \quad x_o[l] \geq 0
\]
- x = avg_price.keys()
- x_o = avg_pax.keys()

#### 8. Binary Constraints

\[
y[(l,k)] is binary
\]

### Retrieved Information
{example_data_description}

### Generated Code

```python
import gurobipy as gp
from gurobipy import GRB
{example_data_description}
total_set = sigma_inflow_A + sigma_outflow_A + sigma_inflow_B + sigma_outflow_B + sigma_inflow_C + sigma_outflow_C
total_set = list(set(total_set)) 
model = gp.Model("sales_based_lp")
y = model.addVars(total_set, vtype=GRB.BINARY, name="y")  
x = model.addVars(avg_price.keys(), lb=0, name="x") 
x_o = model.addVars(value_0_list.keys(), lb=0, name="x_o")  

model.setObjective(gp.quicksum(avg_price[key] * x[key] for key in avg_price.keys()), GRB.MAXIMIZE)

paired_keys = []
for i in range(0, len(avg_price.keys()), 2):  
    if i + 1 < len(avg_price.keys()):  
        names = list(avg_price.keys())
        model.addConstr(
            capacity_consum* x[names[i]] + x[names[i + 1]] <= 187,
            name=f"capacity_constraint"
        )

for l in ratio_0_list.keys():
    temp = 0
    for key in ratio_list.keys():
        if l in key:
            temp += ratio_list[key] * x[key]
    model.addConstr(
        float(ratio_0_list[l]) * x_o[l] + temp == float(avg_pax[l])
    )

for i in value_list.keys():
    for l in ratio_0_list.keys():
        if l in i:
            model.addConstr(
                value_0_list[l]  * x[i] <= value_list[i] * x_o[l]
            )


for key in avg_price.keys():
    for lk in total_set:
       if lk.split(',')[0].strip('(').strip(')')==key.split(',')[0].strip('(').strip(')') and lk.split(',')[1].strip('(').strip(')')==key.split(',')[1].strip('(').strip(')'):
            model.addConstr(
                x[key] <= 10000000 * y[lk],  
                name=f"big_m_constraint"
            )

model.addConstr(gp.quicksum(y[lk] for lk in total_set) <= option_num,
        name=f"cardinality_constraint"
    )


model.addConstr(
    gp.quicksum(y[inflow] for inflow in sigma_inflow_A) == gp.quicksum(y[outflow] for outflow in sigma_outflow_A),
    name=f"flow_conservation_A"
)


model.addConstr(
    gp.quicksum(y[inflow] for inflow in sigma_inflow_B) == gp.quicksum(y[outflow] for outflow in sigma_outflow_B),
    name=f"flow_conservation_B"
)
    
model.addConstr(
    gp.quicksum(y[inflow] for inflow in sigma_inflow_C) == gp.quicksum(y[outflow] for outflow in sigma_outflow_C),
    name=f"flow_conservation_C"
)

model.optimize()


if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(v.varName, v.x)
    print("Optimal objective value:", model.objVal)
else:
    print("No optimal solution found.")
```
'''

    return final_answer

def ProcessPolicyFlow(query):
    output_model = policy_sblp_flow_model_code(query)

    return output_model

## CA

In [ ]:
def csv_qa_tool_CA(query: str,raw_query):
    new_vectors = New_Vectors_Flight(query)
    matches = re.findall(r"\(OD\s*=\s*(\(\s*'[^']+'\s*,\s*'[^']+'\s*\))\s+AND\s+Departure\s*Time\s*=\s*'(\d{1,2}:\d{2})'\)", query)
    capacity_match = re.search(r"Eco_flex ticket consumes (\d+\.?\d*)\s*units", raw_query)

    if capacity_match:
        eco_flex_capacity = capacity_match.group(1)
    else:
        eco_flex_capacity = 1.2

    avg_price, x, x_o, ratio_0_list, ratio_list, value_list, value_0_list, avg_pax = {}, {}, {}, {}, {}, {}, {}, {}

    if matches == []:
        pattern = r"\('(\w+)'\s*,\s*'(\d{1,2}:\d{2})'\)"
        matches_2 = re.findall(pattern, query)
        matches = matches_2
        for od,time in matches:
            origin = od[0]
            destination = od[1]
            od = str((origin, destination))
            code_f = f"({origin}{destination},{time},f)"
            code_l = f"({origin}{destination},{time},l)"
            code_o = f"{origin}{destination}"
            x[code_f] = f"x_{origin}{destination}_{time}_f"
            x[code_l] = f"x_{origin}{destination}_{time}_l"
            retriever = new_vectors.as_retriever(search_kwargs={'k': 1,"filter": {"OD": od, "time": time}})

            doc_1= retriever.invoke(f"OD={od}, Departure Time={time}, Oneway_Product=Eco_flexi, avg_price=")
            for doc in doc_1:
                content = doc.page_content
                pattern = r',\s*(?=\w+=)'
                parts = re.split(pattern, content)

                pairs = [p.strip().replace('"', "'") for p in parts]
                for pair in pairs:
                    key, value = pair.split('=')
                    if key == 'avg_price':
                        avg_price[code_f] = float(value)


            doc_2= retriever.invoke(f"OD={od}, Departure Time={time}, Oneway_Product=Eco_lite, avg_price=")
            for doc in doc_2:
                content = doc.page_content
                pattern = r',\s*(?=\w+=)'
                parts = re.split(pattern, content)

                pairs = [p.strip().replace('"', "'") for p in parts]
                for pair in pairs:
                    key, value = pair.split('=')
                    if key == 'avg_price':
                        avg_price[code_l] =float(value)

            value_1,ratio_1,value_2,ratio_2,value_0,ratio_0 = generate_coefficients(od,time)

            ratio_list[code_f] = ratio_1
            ratio_list[code_l] = ratio_2
            value_list[code_f] = value_1
            value_list[code_l] = value_2
            ratio_0_list[code_o] = ratio_0
            value_0_list[code_o] = value_0
    else:
        for match in matches:
            origin = match[0][2]
            destination = match[0][7]
            time = match[1]
            od = str((origin, destination))
            code_f = f"({origin}{destination},{time},f)"
            code_l = f"({origin}{destination},{time},l)"
            code_o = f"{origin}{destination}"
            x[code_f] = f"x_{origin}{destination}_{time}_f"
            x[code_l] = f"x_{origin}{destination}_{time}_l"

            retriever = new_vectors.as_retriever(search_kwargs={'k': 1,"filter": {"OD": od, "time": time}})
            doc_1= retriever.invoke(f"OD={od}, Departure Time={time}, Oneway_Product=Eco_flexi, avg_price=")
            for doc in doc_1:
                content = doc.page_content
                pattern = r',\s*(?=\w+=)'
                parts = re.split(pattern, content)

                pairs = [p.strip().replace('"', "'") for p in parts]
                for pair in pairs:
                    key, value = pair.split('=')
                    if key == 'avg_price':
                        avg_price[code_f] = float(value)


            doc_2= retriever.invoke(f"OD={od}, Departure Time={time}, Oneway_Product=Eco_lite, avg_price=")
            for doc in doc_2:
                content = doc.page_content
                pattern = r',\s*(?=\w+=)'
                parts = re.split(pattern, content)

                pairs = [p.strip().replace('"', "'") for p in parts]
                for pair in pairs:
                    key, value = pair.split('=')
                    if key == 'avg_price':
                        avg_price[code_l] = float(value)

            value_1,ratio_1,value_2,ratio_2,value_0,ratio_0 = generate_coefficients(od,time)

            ratio_list[code_f] = ratio_1
            ratio_list[code_l] = ratio_2
            value_list[code_f] = value_1
            value_list[code_l] = value_2
            ratio_0_list[code_o] = ratio_0
            value_0_list[code_o] = value_0

    od_matches = re.findall(
    r"OD\s*=\s*\(\s*'([^']+)'\s*,\s*'([^']+)'\s*\)", 
    query
    )

    if od_matches == []:
        pattern = r"\(\('(\w+)','(\w+)'\)"
        matches = re.findall(pattern, query)
        od_matches = matches
    
    od_matches = list(set(od_matches))

    new_vectors_demand = New_Vectors_Demand(query)
    for origin, dest in od_matches:
        od = str((origin, dest))
        code_o = f"{origin}{dest}"
        x_o[code_o] = f"x_{origin}{dest}_o"
        retriever = new_vectors_demand.as_retriever(search_kwargs={'k': 1})
       
        doc_1= retriever.invoke(f"OD={od}, avg_pax=")
        content = doc_1[0].page_content

        pattern = r',\s*(?=\w+=)'
        parts = re.split(pattern, content)
        pairs = [p.strip().replace('"', "'") for p in parts]

        for pair in pairs:
            key, value = pair.split('=')
            if key == 'avg_pax':
                avg_pax[code_o] = value
        
    doc =f"avg_price={avg_price}\n"
    doc +=f"value_list ={value_list}\n"
    doc +=f"ratio_list={ratio_list}\n"
    doc +=f"value_0_list={value_0_list}\n"
    doc +=f"ratio_0_list={ratio_0_list}\n"
    doc +=f"avg_pax={avg_pax}\n"
    doc +=f"capacity_consum = {eco_flex_capacity}\n"
    doc +=f"flight_capacity = 187 \n"
    return doc

def final_tool_CA(problem_description):
    example_matches = retrieve_key_information(problem_description)
    example_data_description = csv_qa_tool_CA(example_matches,problem_description)
    example_data_description = example_data_description.replace('np.float64', 'float')
    example_data_description = example_data_description.replace('np.int64', 'int')
    return example_data_description
def CA_Agent(query):

    loader = CSVLoader(file_path="Large_Scale_Or_Files/RAG_Example_SBLP_CA.csv", encoding="utf-8")
    data = loader.load()
    documents = data
    vectors = FAISS.from_documents(documents, embeddings)
    retriever = vectors.as_retriever(search_kwargs={'k': 1})
    similar_results = retrieve_similar_docs(query,retriever)
    problem_description = similar_results[0]['content'].replace("prompt:", "").strip()   
    example_data_description = final_tool_CA(problem_description)
    fewshot_example = f'''
    Question: {problem_description}

    Thought: I need to retrive the required OD and Departure Time information.

    Action: CSVQA

    Action Input: {problem_description}

    Observation: {example_data_description}

    Thought: I have successfully received the data. I must now return it.
    Final Answer: {example_data_description}
    '''
    tools = [Tool(name="CSVQA", func=final_tool_CA, description="Retrieve flight data.")]
    fewshot_example = fewshot_example.replace('{', '{{')
    fewshot_example = fewshot_example.replace('}', '}}')    

    prefix = (
f"{fewshot_example}\n\n"
"You MUST follow EXACTLY the pattern below for the **FIRST** response.\n"
"Output **ONLY** the following three lines:\n\n"
"Thought: I must first use the CSVQA tool to retrieve necessary data based on the user query.\n"
"Action: CSVQA\n"
"Action Input: {input}\n"
)

    suffix = (
    "### FINAL STEP\n"
    "Use the latest Observation above.\n"
    "Output exactly two lines:\n"
    "Thought: Data retrieval complete. I will now return it.\n"
    "Final Answer: <copy the Observation content above EXACTLY as is>"
)

    agent2 = initialize_agent(
    tools,
    llm=llm2,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={
        "prefix": prefix,
        "suffix": suffix,
    },
    verbose=True,
    handle_parsing_errors=True,
    max_iterations = 5
    )

    return agent2


def get_answer(query):
    agent2 = CA_Agent(query)
    result = agent2.invoke({"input": query})
    output_model = result['output']
    final_answer = f'''### Objective Function

\[
\max \quad \sum_(l,k,j) avg\_price[(l,k,j)] \cdot x[(l,k,j)]
\]

### Constraints

#### 1. Capacity Constraints

For each flight (OD pair \( l \), departure time \( k \)):

\[
capacity\_consum \cdot x[(l,k,f)] + x[(l,k,l)] \leq flight\_capacity
\]

#### 2. Balance Constraints

For each OD pair \( l \):

\[
ratio\_0\_list[l] \cdot x_o[l] + \sum_k \sum_j ratio\_list[(l,k,j)] \cdot x[(l,k,j)] = avg\_pax[l]
\]

#### 3. Scale Constraints

For each ticket option \((l,k,j)\):

\[
x[(l,k,j)]/value\_list[(l,k,j)] - x_o[l]/value\_0\_list[l] \leq 0
\]

#### 4. Nonnegativity Constraints

\[
x[(l,k,j)] \geq 0, \quad x_o[l] \geq 0
\]
- x = avg_price.keys()
- x_o = avg_pax.keys()

### Retrieved Information
{output_model}

### Generated Code

```python
import gurobipy as gp
from gurobipy import GRB
{output_model}
model = gp.Model("sales_based_lp")
x = model.addVars(avg_price.keys(), lb=0, name="x") 
x_o = model.addVars(value_0_list.keys(), lb=0, name="x_o")  

model.setObjective(gp.quicksum(avg_price[key] * x[key] for key in avg_price.keys()), GRB.MAXIMIZE)

paired_keys = []
for i in range(0, len(avg_price.keys()), 2):  
    if i + 1 < len(avg_price.keys()):  
        names = list(avg_price.keys())
        model.addConstr(
            capacity_consum* x[names[i]] + x[names[i + 1]] <= 187,
            name=f"capacity_constraint"
        )

for l in ratio_0_list.keys():
    temp = 0
    for key in ratio_list.keys():
        if l in key:
            temp += ratio_list[key] * x[key]
    model.addConstr(
        float(ratio_0_list[l]) * x_o[l] + temp == float(avg_pax[l])
    )

for i in value_list.keys():
    for l in ratio_0_list.keys():
        if l in i:
            model.addConstr(
                value_0_list[l]  * x[i] <= value_list[i] * x_o[l]
            )

model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(v.varName, v.x)
    print("Optimal objective value:", model.objVal)
else:
    print("No optimal solution found.")
```
Remember:
- Include all parameters in full form.
- Follow the structure and formatting in the example.'''

    return final_answer
def ProcessCA(query):
    CA_model = get_answer(query)
    return CA_model


# Combined Function

In [ ]:
def Process_Input(query):
  category_original = Problemtype(query)
  print(f"Problem type classification finished, it belongs to {category_original}.")
  if "Sales-Based Linear Programming" or "Sales-Based"in    category_original:
    print("Processing AirNRM queries")
    if "flow conservation constraints" in query or "flow conservation constraint" in query:
        print('----------Flow Constraints----------')
        print("Recommend Optimal Flights With Flow Conervation Constraints")
        output_model= ProcessPolicyFlow(query)
        Type = "Policy_Flow"

    else:
        print('----------CA----------')
        print("Only Develop Mathematic Formulations. No Recommendation for Flights.")
        output_model = ProcessCA(query)
        Type = "CA"
  return Type,output_model

# Run the test

In [ ]:
from tqdm import tqdm

def Batch_Process_Queries(df, query_column='Query'):
    """Process in Batches"""
    results = []

    for query in tqdm(df['Query'], desc="Processing Queries"):
        category, output_model = Process_Input(query)
        record = {
            "Category": category,
            "Original_Query": query,
            "Output": output_model,
        }
        results.append(record)

    return pd.DataFrame(results)


In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import re
import numpy as np

def extract_objective(log_text):
   
    if "Optimal objective" in log_text:
        pattern = r"Optimal objective value: ([-+]?\d*\.?\d+)"
        match = re.search(pattern, log_text)
        if match:
            return float(match.group(1))

    elif "Best objective" in log_text:
        pattern = r"Best objective\s+([-+]?\d*\.?\d+e[-+]?\d+)"
        match = re.search(pattern, log_text, re.IGNORECASE)
        if match:
            return float(match.group(1))
    return None  



def gain_obj(df_out):
    output_code = df_out['Code'].tolist()
    obj = []
    i = 1

    for code in output_code:
        print(i)
        namespace = {'gp': gp, 'GRB': GRB}
        
  
        log_output = []
        

        class LogCapture:
            def write(self, message):
                log_output.append(message)

            def flush(self):  
                pass
            
        import sys
        log_capture = LogCapture()
        original_stdout = sys.stdout  
        sys.stdout = log_capture  

        try:
            exec(code, namespace)
        except Exception as e:
            log_output.append(f"Error: {e}\n")
            print(f"Error executing code block {i}: {e}")
        finally:
            sys.stdout = original_stdout 

     
        log_text = ''.join(log_output)

    
        optimal_value = extract_objective(log_text)
        if optimal_value is not None:
            obj.append(optimal_value)
            print(f"Optimal Value: {optimal_value}")
        else:

            obj.append("No optimal value found.")
            print("No optimal value found.")
        i += 1
    return obj


### NP-Test

In [ ]:
df = pd.read_csv('Test_Dataset/Air_NRM/query_NP_Flow.csv')
result_df = Batch_Process_Queries(df)
result_df.to_csv("example_answer_NP_Flow_test_oss_New.csv", index=False)

Code = []
for i in range(len(result_df['Output'])):
    text = result_df['Output'][i]
    pattern = r'```python(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    code = matches[0]
    if 'gurobipy' in code:
        Code.append(code)
    else:
        if len(matches) > 0:
            Code.append(matches[1])
        else:
            Code.append("No code found")

code_df = pd.DataFrame(Code, columns=['Code'])
code_df.to_csv("example_code_NP_Flow_test_oss_New.csv", index=False)

obj = gain_obj(code_df)
obj_df = pd.DataFrame({'Optimal Value': obj})
obj_df.to_csv('OBJ_NP_oss_New.csv', index=False)
combined_df = pd.concat([result_df, code_df,obj_df], axis=1)
combined_df.to_csv("final_answer_NP_Flow_test_oss_New.csv", index=False)

### Run CA

In [ ]:
df = pd.read_csv('Test_Dataset/Air_NRM/query_CA.csv')
result_df = Batch_Process_Queries(df)
result_df.to_csv("answer_CA_test_oss_New.csv", index=False)

Code = []
for i in range(len(result_df['Output'])):
    text = result_df['Output'][i]
    pattern = r'```python(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    code = matches[0]
    if 'gurobipy' in code:
        Code.append(code)
    else:
        if len(matches) > 0:
            Code.append(matches[1])
        else:
            Code.append("No code found")

code_df = pd.DataFrame(Code, columns=['Code'])
code_df.to_csv("code_CA_test_oss_New.csv", index=False)

obj = gain_obj(code_df)
obj_df = pd.DataFrame({'Optimal Value': obj})
obj_df.to_csv('OBJ_CA_oss_New.csv', index=False)
combined_df = pd.concat([result_df, code_df,obj_df], axis=1)
combined_df.to_csv("final_answer_CA_test_oss_New.csv", index=False)

### Run NP Flow

In [ ]:
df = pd.read_csv('Test_Dataset/Air_NRM/query_NP_Flow.csv')
result_df = Batch_Process_Queries(df)
result_df.to_csv("answer_NP_Flow_test_GPT4.1_New_V1.csv", index=False)

Code = []
for i in range(len(result_df['Output'])):
    text = result_df['Output'][i]
    pattern = r'```python(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    code = matches[0]
    if 'gurobipy' in code:
        Code.append(code)
    else:
        if len(matches) > 0:
            Code.append(matches[1])
        else:
            Code.append("No code found")

code_df = pd.DataFrame(Code, columns=['Code'])
code_df.to_csv("code_NP_Flow_test_GPT4.1_New_V1.csv", index=False)

obj = gain_obj(code_df)
obj_df = pd.DataFrame({'Optimal Value': obj})
obj_df.to_csv('OBJ_NP_New_4.1.csv', index=False)

combined_df = pd.concat([result_df, code_df,obj_df], axis=1)
combined_df.to_csv("final_answer_NP_Flow_test_GPT4.1_New_V1.csv", index=False)